In [ ]:
import os
import glob
import numpy as N
from datetime import date, datetime, timedelta
import itertools
import shutil

In [ ]:
basedir = '/Users/dawson29/temp_PIPS/converted/'
converteddatadir = os.path.join(basedir, 'converted_042017_050117')
output_dir = os.path.join(basedir, 'sorted')

In [ ]:
def sort_files(filelist, verbose=False):
    """Sorts a list of converted PIPS files by timestamp. Returns the sorted list and a list of the timestamps"""
    # Read each file in turn and extract the timestamp of the first record, which will be used to sort the files by time
    timestamps = []
    for filename in filelist:
        # print(filename)
        with open(filename, 'r') as f:
            for _ in range(4):
                dummy = f.readline() # I tried using next(f) here but then I get a ValueError on the next line about mixing
                                     # reads and interations. Why?
                #print dummy
            line = f.readline().strip().split(',')
            # print(type(line))
            ts = line[0].strip('"')
            # print(type(ts))
#             if verbose:
#                 print(ts)
#                 print(len(ts))
#                 print(ts[:4], ts[5:7], ts[8:10], ts[11:13], ts[14:16], ts[17:19])
#                 print(ts[0])
# #                 print(int(ts[0]))
# #                 print(int(ts[5:7]))
# #                 print(int(ts[8:10]))
# #                 print(int(ts[11:13]))
# #                 print(int(ts[14:16]))
# #                 print(int(ts[17:19]))
                
            timestamp = datetime(int(ts[:4]), int(ts[5:7]), int(ts[8:10]), int(ts[11:13]), int(ts[14:16]),
                                 int(ts[17:19]))
            timestamps.append(timestamp)

    # Now, sort the file list by the list of timestamps
    # #temp = sorted(zip(filelistraw,filelistconverted), key=lambda pair: os.path.getmtime(pair[0]))
    tuplist = [tup for tup in sorted(zip(timestamps,filelistconverted), key=lambda pair: pair[0])]
    timestamps_sorted = [pair[0] for pair in tuplist]
    filelistconverted_sorted = [pair[1] for pair in tuplist]
    #filelistraw_sorted = [y for (y,x) in sorted(zip(filelistraw,filelistconverted), key=lambda pair: os.path.getctime(pair[0]))]
    #filelistconverted_sorted = [x for (y,x) in sorted(zip(filelistraw,filelistconverted), key=lambda pair: os.path.getctime(pair[0]))]
    #filelistraw_sorted = [x for x in sorted(filelistraw,key=lambda y: os.path.getctime(y))]

    # Check that it's sorted (https://stackoverflow.com/questions/3755136/pythonic-way-to-check-if-a-list-is-sorted-or-not):
    if verbose:
        print(all(timestamps_sorted[i] <= timestamps_sorted[i+1] for i in range(len(timestamps_sorted)-1)))
    
    return timestamps_sorted, filelistconverted_sorted


def organize_files(filelist, timestamps_list, output_dir):
    """Organizes files in subdirectories by date."""
    datelist = [d.date() for d in timestamps_list]
    for key,group in itertools.groupby(zip(datelist, filelist), lambda x: x[0]):
        keystring = key.strftime('%m%d%y')
        keydir = os.path.join(output_dir, keystring)
        # print keystring
        if (not os.path.exists(keydir)):
            os.makedirs(keydir)
        for path in group:
            print(path[1])
            shutil.copy2(path[1],keydir)
    

In [ ]:
filelistconverted = glob.glob(os.path.join(converteddatadir, '*TenHz*.dat'))

In [ ]:
timestamps_sorted, filelistconverted_sorted = sort_files(filelistconverted, verbose=True)

In [ ]:
datelist = [d.date() for d in timestamps_sorted]
print(datelist)
print(filelistconverted_sorted)

In [ ]:
organize_files(filelistconverted_sorted, timestamps_sorted, output_dir)